In [9]:
import os 
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()
openaiAPIkey = os.getenv("OPENAI_API_KEY")

print(openaiAPIkey)

sk-proj-UBi3Ttja20Wnq8FbzG1pT3BlbkFJiFDcAN8Yf3QWE080MsLL


In [10]:
'''
PROBLEM WE ARE SOLVING:
In the usual CoT (Chain of Thoughts) approach, LLMs tend to progress linearly in their thinking towards problem solving, 
and if an error occurs along the way, they tend to proceed along that erroneous criterion.

In the ToT (Tree of Thoughts) approach, LLMs evaluate themselves at each stage of thought and stop inefficient 
approaches early, switching to alternative methods.


My Notes:
- Token generation takes too long
- Too much time to generate tokens and have them be checked each time
- The same llm scoring and assigning confidence level is a terrible idea (Need a better alternative)

'''


'\nPROBLEM WE ARE SOLVING:\nIn the usual CoT (Chain of Thoughts) approach, LLMs tend to progress linearly in their thinking towards problem solving, \nand if an error occurs along the way, they tend to proceed along that erroneous criterion.\n\nIn the ToT (Tree of Thoughts) approach, LLMs evaluate themselves at each stage of thought and stop inefficient \napproaches early, switching to alternative methods.\n\n'

In [11]:
#BrainStorm PHAZE
template ="""
Step1 :
 
I have a problem related to {input}. Could you brainstorm three distinct solutions? Please consider a variety of factors such as {perfect_factors}
A:
"""

prompt = PromptTemplate(
    input_variables=["input","perfect_factors"],
    template = template                      
)

chain1 = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4"),
    prompt=prompt,
    output_key="solutions"
)


In [12]:

#Evaluation PHAZE
template ="""
Step 2:

For each of the three proposed solutions, evaluate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors

{solutions}

A:"""

prompt = PromptTemplate(
    input_variables=["solutions"],
    template = template                      
)

chain2 = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4"),
    prompt=prompt,
    output_key="review"
)


In [13]:
#Expansion Phase

template ="""
Step 3:

For each solution, deepen the thought process. Generate potential scenarios, strategies for implementation, any necessary partnerships or resources, and how potential obstacles might be overcome. Also, consider any potential unexpected outcomes and how they might be handled.

{review}

A:"""

prompt = PromptTemplate(
    input_variables=["review"],
    template = template                      
)

chain3 = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4"),
    prompt=prompt,
    output_key="deepen_thought_process"
)


In [14]:
#Decision Phase

template ="""
Step 4:

Based on the evaluations and scenarios, rank the solutions in order of promise. Provide a justification for each ranking and offer any final thoughts or considerations for each solution
{deepen_thought_process}

A:"""

prompt = PromptTemplate(
    input_variables=["deepen_thought_process"],
    template = template                      
)

chain4 = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4"),
    prompt=prompt,
    output_key="ranked_solutions"
)


In [16]:
#PUTTING IT ALL TOGETHER
from langchain.chains import SequentialChain

#Sequential chain connects the chains together - - - -  The output of one chain becomes the input to the next chain.

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)

print(overall_chain({"input":"human colonization of Mars", "perfect_factors":"The distance between Earth and Mars is very large, making regular resupply difficult"}))



> Entering new SequentialChain chain...

> Finished chain.
{'input': 'human colonization of Mars', 'perfect_factors': 'The distance between Earth and Mars is very large, making regular resupply difficult', 'ranked_solutions': 'Self-Sustaining Colony on Mars:\nPros: This solution would ensure the long-term survival of the colony by making it self-sufficient. It would also be a significant achievement in human space exploration.\nCons: This is a highly ambitious solution that would require significant resources and technological advancements. It would also require a long-term commitment and could face numerous unforeseen challenges.\nInitial Effort: High. This would require significant resources and technological advancements.\nImplementation Difficulty: High. This is a highly ambitious solution that would require a long-term commitment and could face numerous unforeseen challenges.\nPotential Challenges: Developing the necessary technology, securing the necessary funding and resources